<a href="https://colab.research.google.com/github/Guitarjourneyman/Object-detection/blob/main/portholeDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="ccZy59bvyQmmgjjnMEg2")
project = rf.workspace("umat").project("pothole-abpqz")
version = project.version(1)
dataset = version.download("yolov5")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 69.2 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Pothole-1 in yolov5pytorch:: 100%|██████████| 1122/1122 [00:00<00:00, 7638.51it/s]


# Task
Train and test a YOLOv5 model using the dataset located in "content/Prthole-1/train" and "content/Prthole-1/test".

## Install dependencies

### Subtask:
Install the required libraries for YOLOv5.


**Reasoning**:
Clone the YOLOv5 repository and install the required libraries.



In [3]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17582, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 17582 (delta 0), reused 0 (delta 0), pack-reused 17577 (from 1)
Receiving objects: 100% (17582/17582), 16.81 MiB | 9.24 MiB/s, done.
Resolving deltas: 100% (11970/11970), done.
/content/yolov5/yolov5


## Prepare data

### Subtask:
Ensure the dataset is in the correct format and location for YOLOv5 training.


**Reasoning**:
Verify the dataset directory structure and content to ensure it's in the correct format for YOLOv5 and located in the specified content directory.



In [6]:
import os
import glob

# Define the expected dataset path
dataset_path = '/content/Pothole-1' # Correct path relative to the current working directory (yolov5)

# Verify the dataset directory exists
if not os.path.exists(dataset_path):
    print(f"Error: Dataset directory not found at {dataset_path}")
else:
    print(f"Dataset directory found at {dataset_path}")

    # Define expected subdirectories
    expected_subdirs = ['train', 'test']
    for subdir in expected_subdirs:
        subdir_path = os.path.join(dataset_path, subdir)
        if not os.path.exists(subdir_path):
            print(f"Error: Expected subdirectory not found: {subdir_path}")
        else:
            print(f"Subdirectory found: {subdir_path}")
            # Verify images and labels directories within each split
            image_dir = os.path.join(subdir_path, 'images')
            label_dir = os.path.join(subdir_path, 'labels')

            if not os.path.exists(image_dir):
                print(f"Error: Images directory not found: {image_dir}")
            else:
                print(f"Images directory found: {image_dir}")

            if not os.path.exists(label_dir):
                print(f"Error: Labels directory not found: {label_dir}")
            else:
                print(f"Labels directory found: {label_dir}")

            # Check for corresponding image and label files
            image_files = glob.glob(os.path.join(image_dir, '*'))
            label_files = glob.glob(os.path.join(label_dir, '*'))

            print(f"Found {len(image_files)} image files in {image_dir}")
            print(f"Found {len(label_files)} label files in {label_dir}")

            # Basic check for file correspondence (by count and name prefix)
            if len(image_files) != len(label_files):
                 print(f"Warning: Number of images and labels in {subdir_path} do not match.")
            else:
                print(f"Number of images and labels match in {subdir_path}.")

            # Further check by comparing base names
            image_basenames = set([os.path.splitext(os.path.basename(f))[0] for f in image_files])
            label_basenames = set([os.path.splitext(os.path.basename(f))[0] for f in label_files])

            if image_basenames != label_basenames:
                 print(f"Warning: Basenames of images and labels in {subdir_path} do not match.")
            else:
                print(f"Basenames of images and labels match in {subdir_path}.")


    # Check a sample label file format (optional, but good practice)
    sample_label_file = None
    for subdir in expected_subdirs:
        label_files = glob.glob(os.path.join(dataset_path, subdir, 'labels', '*.txt'))
        if label_files:
            sample_label_file = label_files[0]
            break

    if sample_label_file:
        print(f"Checking sample label file: {sample_label_file}")
        with open(sample_label_file, 'r') as f:
            lines = f.readlines()
            if lines:
                first_line = lines[0].strip().split()
                # Expecting at least 5 values: class_id, center_x, center_y, width, height
                if len(first_line) >= 5:
                    try:
                        # Check if the values are convertible to float
                        _ = [float(val) for val in first_line]
                        print("Sample label file appears to be in correct YOLO format.")
                    except ValueError:
                        print("Error: Sample label file does not contain valid numeric values.")
                else:
                    print("Error: Sample label file does not have enough columns (expected at least 5).")
            else:
                print("Warning: Sample label file is empty.")
    else:
        print("No label files found to check format.")


Dataset directory found at /content/Pothole-1
Subdirectory found: /content/Pothole-1/train
Images directory found: /content/Pothole-1/train/images
Labels directory found: /content/Pothole-1/train/labels
Found 389 image files in /content/Pothole-1/train/images
Found 389 label files in /content/Pothole-1/train/labels
Number of images and labels match in /content/Pothole-1/train.
Basenames of images and labels match in /content/Pothole-1/train.
Subdirectory found: /content/Pothole-1/test
Images directory found: /content/Pothole-1/test/images
Labels directory found: /content/Pothole-1/test/labels
Found 55 image files in /content/Pothole-1/test/images
Found 55 label files in /content/Pothole-1/test/labels
Number of images and labels match in /content/Pothole-1/test.
Basenames of images and labels match in /content/Pothole-1/test.
Checking sample label file: /content/Pothole-1/train/labels/images199_jpg.rf.4df543b87f7568b315eac958ca01e4b1.txt
Sample label file appears to be in correct YOLO f

## Configure Training

### Subtask:
Create a YAML configuration file for YOLOv5 training.

**Reasoning**:
Create a `data.yaml` file in the YOLOv5 directory. This file will contain the paths to the training and testing images and labels, as well as the number of classes and their names.

In [8]:
import yaml
import os

# Define the path to the dataset
dataset_path = '/content/Pothole-1'

# Define the path where the data.yaml file will be saved (within the yolov5 directory)
yaml_path = '/content/yolov5/data/pothole_data.yaml'

# Define the number of classes and class names
nc = 1 # Number of classes (pothole)
names = ['pothole'] # Class names

# Create the data dictionary
data = {
    'train': os.path.join(dataset_path, 'train', 'images'),
    'val': os.path.join(dataset_path, 'test', 'images'), # Using test for validation as per the task
    'nc': nc,
    'names': names
}

# Write the data dictionary to the YAML file
with open(yaml_path, 'w') as f:
    yaml.dump(data, f)

print(f"Created data.yaml file at: {yaml_path}")

Created data.yaml file at: /content/yolov5/data/pothole_data.yaml


## Train Model

### Subtask:
Train the YOLOv5 model using the configured dataset.

**Reasoning**:
Run the YOLOv5 training script with the `pothole_data.yaml` file.

## Evaluate Model

### Subtask:
Run the YOLOv5 testing script to evaluate the trained model.

**Reasoning**:
Run the YOLOv5 validation script (`val.py`) with the trained weights and the data configuration file.

In [12]:
!python val.py --weights runs/train/exp/weights/best.pt --data /content/yolov5/data/pothole_data.yaml --img 640 --task val

val: data=/content/yolov5/data/pothole_data.yaml, weights=['runs/train/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-432-g725b922e Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/Pothole-1/test/labels.cache... 55 images, 0 backgrounds, 0 corrupt: 100% 55/55 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 2/2 [00:02<00:00,  1.30s/it]
                   all         55        193      0.822      0.837      0.861      0.645
Speed: 0.4ms pre-process, 10.3ms inference, 4.6ms NMS per image at shape (32, 3, 640, 640)
Results saved to r

## Finish task

Summarize the results and provide insights into the model's performance.